<a href="https://colab.research.google.com/github/ErnestoCash/ASAP/blob/main/EmDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report
from flask import Flask, request, jsonify

# Improved random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Generating a more varied synthetic dataset for emotion analysis
data_size = 1000  # Increased number of data points for better training
emotions = ['happy', 'angry', 'sad', 'neutral', 'excited', 'disappointed']  # Added more emotions
feedback_examples = [
    "I love this product, it's absolutely fantastic!",
    "This is the worst experience I've ever had.",
    "I'm so sad about this service, it didn't meet my expectations.",
    "The product is fine, nothing special though.",
    "Wow! This exceeded my expectations, so excited!",
    "Quite disappointing, I had higher expectations."
]

# Creating the dataset with more examples
data = {
    "Feedback": [random.choice(feedback_examples) for _ in range(data_size)],
    "Emotion": [random.choice(emotions) for _ in range(data_size)]
}

# Creating a DataFrame
feedback_df = pd.DataFrame(data)

# Displaying the first few rows of the dataset
print(feedback_df.head())

# Text Vectorization and Label Encoding
tfidf = TfidfVectorizer(max_features=2500)  # Increased max_features for better text representation
X = tfidf.fit_transform(feedback_df['Feedback']).toarray()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(feedback_df['Emotion'])

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Neural Network Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),  # Added an extra layer
    Dense(len(label_encoder.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))  # Increased epochs

# Model Evaluation
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes))

# Save the model
model.save('emotion_model.h5')

# Flask App for Model Deployment
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    model = load_model('emotion_model.h5')  # Load the trained model
    text = request.json.get("text", "")
    features = tfidf.transform([text]).toarray()
    prediction = model.predict(features)
    emotion = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    return jsonify({"emotion": emotion})

if __name__ == '__main__':
    app.run(debug=True)  # Run in debug mode for development

# Note: Removed flask-ngrok as it's generally used for Colab notebooks.
# For deployment, consider using a proper web server.




                                            Feedback  Emotion
0    Quite disappointing, I had higher expectations.  excited
1    I love this product, it's absolutely fantastic!  neutral
2    I love this product, it's absolutely fantastic!  excited
3    Quite disappointing, I had higher expectations.  excited
4  I'm so sad about this service, it didn't meet ...  excited
Epoch 1/15
25/25 [==============================] - 5s 52ms/step - loss: 1.7996 - accuracy: 0.1500 - val_loss: 1.7982 - val_accuracy: 0.1800
Epoch 2/15
25/25 [==============================] - 0s 18ms/step - loss: 1.7869 - accuracy: 0.1975 - val_loss: 1.8063 - val_accuracy: 0.1400
Epoch 3/15
25/25 [==============================] - 1s 20ms/step - loss: 1.7849 - accuracy: 0.1963 - val_loss: 1.8044 - val_accuracy: 0.1600
Epoch 4/15
25/25 [==============================] - 1s 29ms/step - loss: 1.7847 - accuracy: 0.1925 - val_loss: 1.8014 - val_accuracy: 0.1850
Epoch 5/15
25/25 [==============================] - 0s 19ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/engine/trainin